# MusicExpander Prototype 1:

In [1]:
#workflow:
#1 .Get a list of hot songs "hotlist" (Functions: create_raw_db, load_df)
#2. Get song name form user: "input song" (Function: get_input)
#3. Search the hotlist for the input song (Function: search_hotlist)
#4. if function search_hostlist returns True, call a function that retrieves another hot song (Function: Song_retriever)
#5. if function search_hostlist returns False, print a sad message

## Import libraries

In [8]:
#import libraries
from bs4 import BeautifulSoup
import pandas as pd
import requests
from random import randrange
import spotipy # install if needed
from spotipy.oauth2 import SpotifyClientCredentials

## Add YOUR Spotity credentials

In [9]:
#Initialize SpotiPy with user credentials
# Remove mine when uploading

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id="9f9d46aed10c480b9b68543a2d15e2c9",
    client_secret="0016f56c1cc641159dc9e19ba85bc682"))

## Functions:

In [10]:
#functions:

def create_raw_db():
    '''
    Use requests to get html from hot songs site.
    Use BeautifulSoup to parse the html
    Return soup
    '''
    
    url="https://www.billboard.com/charts/hot-100"
    response=requests.get(url)
    
    #if response.status_code !=200 do soemthing?
    
    #create the soup 
    soup=BeautifulSoup(response.content, "html.parser")
    
    return soup
    


def load_df(soup):
    '''
    input: soup from create_raw_db:
    store song names and artist names from the soup
    create a dictionary
    load a pandas df
    '''
    
    #create empty lists to append songs names and artist names
    song_names=[]
    artists_names=[]
    
    #use Beautiful Soup to extract the song names and artists names:
    names_songs=soup.select("span.chart-element__information__song.text--truncate.color--primary")
    names_artists=soup.select("span.chart-element__information__artist.text--truncate.color--secondary")
    
    #loop to append
    for s in names_songs:
        song_names.append(s.get_text())
    
    for s in names_artists:
        artists_names.append(s.get_text())

    #store lists in dictionary
    songs_dict={
        "song titles":song_names,
        "artists":artists_names,
        }
    # Each key becomes a dataframe column name:
    hot_songs_df=pd.DataFrame(songs_dict)
    
    return hot_songs_df

def answer_refiner():
    
    '''
    This is a more complex version of get_input_song
    it asks the user if their song was played by an artist
    if it is the while loop finishes
    if it is not, the loop continues asking questions
    either until the user says it is the right artist
    or until the user quits.
    '''
    #define affirmative and negative answers:
    affirmative=["Yes","YES","yes","yeah","yep","y","Y"]
    negative=["No","NO","no","Nope","NOPE","nope","n","N"]
    attempts=0
    
    #make sure it is the right song:
    status=False
    while status == False:
    
        input_song=input("Tell me the name of a song you like!")
        second_input=input("Do you mean to the song "+input_song+ " by "+ artist_retriver(input_song))
    
        if second_input in affirmative :
            print("Cool! I know which song you mean!")
            status=True

        elif second_input in negative and attempts ==0:
            print("ok, yours must be another song. Let's try again:")
            attempts+=1
            
        elif second_input in negative and attempts >0:
            third_input=input("Do you want to quit?")
            attempts+=1
            if third_input in affirmative:
                print("ok, let's stop, I don't know the songs you like")
                break
            elif third_input in negative:
                print("ok, yours must be another song. Let's try again:")
            else:
                print("Sorry, your answer must be \"Yes\" or \"No\". Try again, please")  
        else:
            print("Sorry, your answer must be \"Yes\" or \"No\". Try again, please")

    return input_song  

def get_input_song():
    input_song=input("Tell me the name of a song you like!")
    return input_song


def search_hotlist(input_song, hot_song_df):
    if len(hot_df.loc[hot_song_df['song titles'] == input_song])!=0:
        return True
        
    else:
        return False
    
def song_retriver(input_song, hot_song_df):
    '''
    get a random song form the sd of hot songs
    '''
    
    #get random number within hot_song_df range:
    numb_r=randrange(len(hot_song_df))
    #song_titles are in column 0:
    #get value form "song title" column and numb_r index:
    return(hot_df.iloc[numb_r,0])


def artist_retriver(input_song):
    '''
    retrieve the artist of a given song with sp.search
    spotify credentials must be provided above
    '''
    #search in spotify the info about the input song
    song_info = sp.search(input_song,limit=1)
    #save the artist's name
    artists_names=(song_info["tracks"]["items"][0]["artists"][0]["name"])
    return artists_names



In [11]:
#main:

#create db
hot_soup=create_raw_db()

#fill a pandas dataframe with the db from BeautifulSoup
hot_df=load_df(hot_soup)

#get a song from the user (make sure it is the right song)
user_song=answer_refiner()


if search_hotlist(user_song, hot_df)== True:
    print("You have a trendy taste! You may also like this other song:")
    new_song=song_retriver(user_song, hot_df)
    print(new_song+" by "+ artist_retriver(new_song))
    
else:
    print("Your taste is not up to date... I can't recommend you new songs, sorry")


Tell me the name of a song you like!soldadet
Do you mean to the song soldadet by Fito y Fitipaldisno
ok, yours must be another song. Let's try again:
Tell me the name of a song you like!Soldadet
Do you mean to the song Soldadet by Fito y Fitipaldisnope
Do you want to quit?no
ok, yours must be another song. Let's try again:
Tell me the name of a song you like!Soldadet
Do you mean to the song Soldadet by Fito y Fitipaldisla cançó del soldadet
Sorry, your answer must be "Yes" or "No". Try again, please
Tell me the name of a song you like!la cançó del soldadet
Do you mean to the song la cançó del soldadet by Manelyes
Cool! I know which song you mean!
Your taste is not up to date... I can't recommend you new songs, sorry
